---
# 상품 discription 속성을 통한 상품 유사도 필터링
### 목적 : 검색 시 유사 상품 출력 

### 데이터셋 walmart 소비자 구매데이터
- StockCode: 구매별 코드 구분
- Description: 상품 설명
- UnitPrice: 상품 가격
- CustomerID: 고객ID
- Country: 구매국가
- Quantity: 구매수량
---

In [40]:
#모듈로딩
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
#데이터 로딩
product_df = pd.read_excel("Online Retail.xlsx")

In [151]:
#결측치 확인
product_copy_df = product_df.copy()
product_copy_df.drop(['InvoiceNo','InvoiceDate'], axis=1,inplace=True)
product_copy_df.isnull().sum()

StockCode           0
Description      1454
Quantity            0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [152]:
#결측치 처리
print(product_copy_df['Description'].isnull().sum())
x = product_copy_df[product_copy_df['Description'].isnull() ==True]
print(x.CustomerID.isnull().sum())
#CustomerID는 결측치 처리를 Imputer를 사용해서 하기에는 애매함 drop
product_copy_df.dropna(inplace=True)
#결측치 처리 완료
product_copy_df.reset_index(inplace=True)
product_copy_df.drop('index',axis = 1,inplace=True)
product_copy_df.info()

1454
1454
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406829 entries, 0 to 406828
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   StockCode    406829 non-null  object 
 1   Description  406829 non-null  object 
 2   Quantity     406829 non-null  int64  
 3   UnitPrice    406829 non-null  float64
 4   CustomerID   406829 non-null  float64
 5   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 18.6+ MB


In [153]:
# 상품데이터 설명 split
product_copy_df['Description_literal'] = product_copy_df['Description'].apply(lambda x: x.split(' '))


In [154]:
Quantity_sum = product_copy_df.groupby('StockCode').sum('Quantity')
Quantity_sum = Quantity_sum.reset_index()
Quantity_sum

,StockCode,Quantity,UnitPrice,CustomerID
0,10002,823,41.650,723842.0
1,10080,291,8.650,333014.0
2,10120,193,6.300,484879.0
3,10125,1226,50.100,976545.0
4,10133,2374,65.830,1942666.0
...,...,...,...,...
3679,D,-1194,5581.310,1149366.0
3680,DOT,16,11906.360,225536.0
3681,M,3184,155507.920,7102962.0
3682,PADS,4,0.003,61901.0


In [155]:

Quantity_sum = Quantity_sum[['StockCode', 'Quantity']]


In [156]:
#중복 제거
product_copy_df= product_copy_df.drop_duplicates(subset=["StockCode"])
product_copy_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3684 entries, 0 to 406349
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   StockCode            3684 non-null   object 
 1   Description          3684 non-null   object 
 2   Quantity             3684 non-null   int64  
 3   UnitPrice            3684 non-null   float64
 4   CustomerID           3684 non-null   float64
 5   Country              3684 non-null   object 
 6   Description_literal  3684 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 230.2+ KB


In [157]:
product_copy_df = product_copy_df.drop('Quantity',axis=1)
product_copy_df = pd.merge(product_copy_df, Quantity_sum, on= 'StockCode')

In [161]:
product_copy_df

,StockCode,Description,UnitPrice,CustomerID,Country,Description_literal,Quantity
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,17850.0,United Kingdom,"[WHITE, HANGING, HEART, T-LIGHT, HOLDER]",34204
1,71053,WHITE METAL LANTERN,3.39,17850.0,United Kingdom,"[WHITE, METAL, LANTERN]",1600
2,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,17850.0,United Kingdom,"[CREAM, CUPID, HEARTS, COAT, HANGER]",1876
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,17850.0,United Kingdom,"[KNITTED, UNION, FLAG, HOT, WATER, BOTTLE]",2455
4,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,17850.0,United Kingdom,"[RED, WOOLLY, HOTTIE, WHITE, HEART.]",3688
...,...,...,...,...,...,...,...
3679,90214U,"LETTER ""U"" BLING KEY RING",0.29,13790.0,United Kingdom,"[LETTER, ""U"", BLING, KEY, RING]",12
3680,90214W,"LETTER ""W"" BLING KEY RING",0.29,13790.0,United Kingdom,"[LETTER, ""W"", BLING, KEY, RING]",12
3681,90214Z,"LETTER ""Z"" BLING KEY RING",0.29,13790.0,United Kingdom,"[LETTER, ""Z"", BLING, KEY, RING]",12
3682,90089,PINK CRYSTAL SKULL PHONE CHARM,0.19,17914.0,United Kingdom,"[PINK, CRYSTAL, SKULL, PHONE, CHARM]",25


In [162]:
product_copy_df['Country'] = product_copy_df['Country'].astype('category')
product_copy_df['Country'].value_counts()

United Kingdom     3396
EIRE                 66
Germany              57
France               47
Norway               33
Australia            12
Japan                11
Italy                11
Portugal              8
Spain                 6
Netherlands           5
Cyprus                5
Belgium               5
Switzerland           4
Lithuania             4
Finland               2
Malta                 2
Iceland               2
Unspecified           2
Denmark               1
Canada                1
Poland                1
Singapore             1
Channel Islands       1
Austria               1
Name: Country, dtype: int64

In [164]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# description_mat = count_vect.fit_transform(product_copy_df['Description'])
# print(description_mat)
count_vect.fit(product_copy_df['Description'])
description_mat = count_vect.transform(product_copy_df['Description'])


In [165]:
print(description_mat.shape)

(3684, 8799)


In [180]:
product_sim = cosine_similarity(description_mat,description_mat)


In [187]:
product_sim_sorted_ind = product_sim.argsort()[:,::-1]
product_sim_sorted_ind

(3684, 3684)

In [188]:
similarity = np.round(np.sort(product_sim)[:,::-1],3)
similarity

array([[1.   , 0.778, 0.745, ..., 0.   , 0.   , 0.   ],
       [1.   , 0.447, 0.4  , ..., 0.   , 0.   , 0.   ],
       [1.   , 0.192, 0.192, ..., 0.   , 0.   , 0.   ],
       ...,
       [1.   , 1.   , 1.   , ..., 0.   , 0.   , 0.   ],
       [1.   , 0.667, 0.504, ..., 0.   , 0.   , 0.   ],
       [1.   , 0.429, 0.21 , ..., 0.   , 0.   , 0.   ]])

In [182]:

product_sim_sorted_ind[:1]

array([[   0,   58,  504, ..., 2338, 2336, 1841]], dtype=int64)

In [169]:
product_copy_df

,StockCode,Description,UnitPrice,CustomerID,Country,Description_literal,Quantity
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,17850.0,United Kingdom,"[WHITE, HANGING, HEART, T-LIGHT, HOLDER]",34204
1,71053,WHITE METAL LANTERN,3.39,17850.0,United Kingdom,"[WHITE, METAL, LANTERN]",1600
2,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,17850.0,United Kingdom,"[CREAM, CUPID, HEARTS, COAT, HANGER]",1876
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,17850.0,United Kingdom,"[KNITTED, UNION, FLAG, HOT, WATER, BOTTLE]",2455
4,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,17850.0,United Kingdom,"[RED, WOOLLY, HOTTIE, WHITE, HEART.]",3688
...,...,...,...,...,...,...,...
3679,90214U,"LETTER ""U"" BLING KEY RING",0.29,13790.0,United Kingdom,"[LETTER, ""U"", BLING, KEY, RING]",12
3680,90214W,"LETTER ""W"" BLING KEY RING",0.29,13790.0,United Kingdom,"[LETTER, ""W"", BLING, KEY, RING]",12
3681,90214Z,"LETTER ""Z"" BLING KEY RING",0.29,13790.0,United Kingdom,"[LETTER, ""Z"", BLING, KEY, RING]",12
3682,90089,PINK CRYSTAL SKULL PHONE CHARM,0.19,17914.0,United Kingdom,"[PINK, CRYSTAL, SKULL, PHONE, CHARM]",25


In [228]:
def find_sim_product(df, sorted_ind, similarity,product_description, top_n = 10):
    product_description = df[df['Description'] == product_description]
    product_index = product_description.index.values
    similar_indexes = sorted_ind[product_index,:(top_n)]
                    
    similar_indexes = similar_indexes.reshape(-1)
    a = similarity[product_index,:(top_n)]
    a= list(a)
    a.split(' ')
    print(a)
    new_df = df.iloc[similar_indexes].copy()
    # new_df['similarity'] = similarity[product_index,:(top_n)]
    return new_df

In [229]:
similar_product = find_sim_product(product_copy_df, product_sim_sorted_ind, similarity,'CLASSIC WHITE FRAME',10)
similar_product

AttributeError: 'list' object has no attribute 'split'